In [1]:
import os
import pandas as pd
from catboost import CatBoostClassifier
import numpy as np
from sklearn.model_selection import GridSearchCV
import time
import joblib
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.evaluate_model import evaluate_model

In [2]:
X_train = pd.read_csv('../data/train_features.csv')
X_test = pd.read_csv('../data/test_features.csv')

In [3]:
X_train

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,glucose_test,a1ctest,change,...,diag_2_other,diag_2_respiratory,diag_3_circulatory,diag_3_diabetes,diag_3_digestive,diag_3_injury,diag_3_missing,diag_3_musculoskeletal,diag_3_other,diag_3_respiratory
0,0.615385,0.580357,0.166667,0.333333,0.000000,0.000000,0.0,0.0,1.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076923,0.339286,0.166667,0.115385,0.000000,0.066667,0.0,0.0,0.0,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.196429,0.000000,0.025641,0.000000,0.000000,0.0,0.0,2.0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.307692,0.473214,0.666667,0.256410,0.000000,0.000000,0.0,0.0,1.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.230769,0.473214,0.000000,0.102564,0.000000,0.066667,0.0,0.0,2.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17495,0.461538,0.383929,0.000000,0.333333,0.148148,0.200000,0.0,0.0,0.0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
17496,0.230769,0.544643,0.000000,0.410256,0.000000,0.133333,0.0,0.0,0.0,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
17497,0.153846,0.107143,0.166667,0.397436,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
17498,0.538462,0.508929,0.666667,0.217949,0.000000,0.000000,0.0,0.0,2.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
X_test

,time_in_hospital,n_lab_procedures,n_procedures,n_medications,n_outpatient,n_inpatient,n_emergency,glucose_test,a1ctest,change,...,diag_2_other,diag_2_respiratory,diag_3_circulatory,diag_3_diabetes,diag_3_digestive,diag_3_injury,diag_3_missing,diag_3_musculoskeletal,diag_3_other,diag_3_respiratory
0,0.000000,0.303571,0.000000,0.076923,0.037037,0.133333,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.076923,0.080357,1.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.846154,0.517857,0.166667,0.179487,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.076923,0.276786,0.000000,0.230769,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.000000,0.419643,0.000000,0.217949,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7495,0.230769,0.303571,0.000000,0.179487,0.000000,0.000000,0.0,0.0,0.0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7496,0.307692,0.348214,0.166667,0.141026,0.000000,0.000000,0.0,0.0,0.0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7497,0.230769,0.410714,0.500000,0.256410,0.000000,0.133333,0.0,0.0,0.0,0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
7498,0.153846,0.339286,0.000000,0.141026,0.000000,0.000000,0.0,0.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
y_train = pd.read_csv('../data/train_labels.csv').squeeze()
y_test = pd.read_csv('../data/test_labels.csv').squeeze()

In [6]:
y_train

0        0
1        1
2        0
3        0
4        0
        ..
17495    0
17496    0
17497    0
17498    0
17499    1
Name: readmission_status, Length: 17500, dtype: int64

In [7]:
y_test

0       1
1       0
2       0
3       0
4       1
       ..
7495    0
7496    1
7497    1
7498    1
7499    0
Name: readmission_status, Length: 7500, dtype: int64

In [8]:
model = CatBoostClassifier( 
    eval_metric='AUC',                # CatBoost uses 'AUC' or 'Logloss' as strings
    loss_function='Logloss',          # Standard loss function for binary classification
    random_state=42,
    verbose=0                         # Suppresses training output for a cleaner notebook
)

In [9]:
print("Training CatBoost model...")
model.fit(X_train, y_train)
print("Training complete.")

Training CatBoost model...
Training complete.


In [10]:
y_pred_class = model.predict(X_test)

In [11]:
y_pred_class

array([1, 0, 0, ..., 1, 0, 1], shape=(7500,))

In [12]:
y_pred_class

array([1, 0, 0, ..., 1, 0, 1], shape=(7500,))

In [13]:
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [14]:
y_pred_proba

array([0.58920769, 0.30236024, 0.37911835, ..., 0.62937455, 0.45519573,
       0.58263246], shape=(7500,))

In [15]:
result = evaluate_model(model, X_test, y_test, "CatBoostClassifier", False)
print(result)

{'model_name': 'CatBoostClassifier', 'Accuracy': 0.6164, 'Precision': 0.6067146282973621, 'Recall (Sensitivity)': 0.506, 'F1_Score': 0.5517993456924755, 'ROC_AUC': 0.6591479285714286, 'Confusion_Matrix_Values': {'True Negatives (TN)': 2852, 'False Positives (FP)': 1148, 'False Negatives (FN)': 1729, 'True Positives (TP)': 1771}}


In [16]:
y_train[y_train == 0].count() / y_train.count()

np.float64(0.5283428571428571)

In [17]:
y_train[y_train == 1].count() / y_train.count()

np.float64(0.47165714285714283)

In [18]:
neg_count = np.sum(y_train == 0)
pos_count = np.sum(y_train == 1)
scale_pos_weight_value = neg_count / pos_count
print(f"Calculated scale_pos_weight: {scale_pos_weight_value:.2f}")

Calculated scale_pos_weight: 1.12


In [19]:
param_grid = {
    # Number of trees (same as n_estimators in XGBoost)
    'iterations': [100, 200, 300], 
    
    # Maximum depth of the tree (CatBoost's equivalent of max_depth)
    'depth': [3, 5, 7], 
    
    # Learning rate (controls step size)
    'learning_rate': [0.01, 0.05, 0.1], 
    
    # Weight to apply to the positive class (readmission). 
    # This is CatBoost's way of handling imbalance, similar to scale_pos_weight.
    'scale_pos_weight': [scale_pos_weight_value]
}

In [20]:
start_time = time.time()
print("\nStarting GridSearchCV...")

gscv = GridSearchCV(
    # Base Estimator: XGBoost Classifier (with fixed parameters)
    estimator=CatBoostClassifier( 
        eval_metric='AUC',
        loss_function='Logloss',
        random_state=42,
        verbose=0
    ),
    
    # Search Space: The dictionary of hyperparameters
    param_grid=param_grid,
    
    # Scoring Metric: Use the AUC-ROC scorer defined above
    scoring='roc_auc',
    
    # Cross-Validation Folds: 5-fold is standard
    cv=5,
    
    # Verbosity: Shows progress during the search
    verbose=3,
    
    # Number of cores to use: -1 uses all available cores for speed
    n_jobs=-1
)

# This step trains hundreds of models based on the combinations in param_grid
gscv.fit(X_train, y_train)

end_time = time.time()
print(f"\nGrid Search completed in {(end_time - start_time) / 60:.2f} minutes.")


Starting GridSearchCV...
Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 4/5] END depth=3, iterations=100, learning_rate=0.01, scale_pos_weight=1.1201841531378725;, score=0.644 total time=   2.1s
[CV 1/5] END depth=3, iterations=100, learning_rate=0.01, scale_pos_weight=1.1201841531378725;, score=0.637 total time=   2.5s
[CV 3/5] END depth=3, iterations=100, learning_rate=0.05, scale_pos_weight=1.1201841531378725;, score=0.657 total time=   2.5s
[CV 3/5] END depth=3, iterations=100, learning_rate=0.01, scale_pos_weight=1.1201841531378725;, score=0.646 total time=   2.9s
[CV 2/5] END depth=3, iterations=100, learning_rate=0.05, scale_pos_weight=1.1201841531378725;, score=0.651 total time=   2.8s
[CV 2/5] END depth=3, iterations=100, learning_rate=0.01, scale_pos_weight=1.1201841531378725;, score=0.645 total time=   4.3s
[CV 5/5] END depth=3, iterations=100, learning_rate=0.05, scale_pos_weight=1.1201841531378725;, score=0.648 total time=   2.6s
[CV 4/5] END depth=3, i

In [21]:
print("\n--- Grid Search Results ---")
print(f"Best AUC-ROC Score achieved: {gscv.best_score_:.4f}")
print("Best Hyperparameters found:")
print(gscv.best_params_)

# Retrieve the best model object
best_catboost_model = gscv.best_estimator_


--- Grid Search Results ---
Best AUC-ROC Score achieved: 0.6591
Best Hyperparameters found:
{'depth': 5, 'iterations': 300, 'learning_rate': 0.05, 'scale_pos_weight': np.float64(1.1201841531378725)}


Testing the best fit model on test data

In [22]:
y_test_proba = best_catboost_model.predict_proba(X_test)[:, 1]
y_test_pred_class = best_catboost_model.predict(X_test)

In [23]:
print('\nBest CatBoost Model Evaluation on Test Set:')
result = evaluate_model(best_catboost_model, X_test, y_test, "CatBoostClassifier", True)
print(result)


Best CatBoost Model Evaluation on Test Set:
Metrics for CatBoostClassifier saved/updated in /media/anubhaw/Anubhaw/Projects/Hospital Readmission Prediction/results.json
{'model_name': 'CatBoostClassifier', 'Accuracy': 0.6168, 'Precision': 0.5987381703470032, 'Recall (Sensitivity)': 0.5422857142857143, 'F1_Score': 0.5691154422788606, 'ROC_AUC': 0.6616489999999999, 'Confusion_Matrix_Values': {'True Negatives (TN)': 2728, 'False Positives (FP)': 1272, 'False Negatives (FN)': 1602, 'True Positives (TP)': 1898}}


### Saving the model

In [24]:
try:
    os.makedirs('../models', exist_ok=True)
except Exception as e:
    print(f"An error occurred: {e}")

In [25]:
joblib.dump(best_catboost_model, '../models/best_catboost_model.joblib')

['../models/best_catboost_model.joblib']